# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [62]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [63]:
# Load the CSV file created in Part 1 into a Pandas DataFrame 
city_data_df = pd.read_csv("..\output_data\cities.csv") 

# Display sample data 
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,269.32,81,100,2.06,RU,1715487759
1,1,lazaro cardenas,17.9583,-102.2000,296.55,81,61,1.05,MX,1715487765
2,2,hasaki,35.7333,140.8333,297.20,71,100,11.54,JP,1715487765
3,3,naze,28.3667,129.4833,298.66,76,70,4.76,JP,1715487766
4,4,ilanskiy,56.2358,96.0650,284.91,53,100,6.11,RU,1715487767


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [64]:
import hvplot.pandas

# Define colors
colors = ['#FFA500', '#8B4513', '#0000FF', '#008000']  # Orange, Brown, Blue, Green

# Create a scatter plot using hvplot with predefined colors
scatter_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    size='Humidity',
    hover_cols=['City'],
    title='City Humidity',
    xlabel='Longitude',
    ylabel='Latitude',
    frame_width=700,
    frame_height=500,
    color=colors,  # Predefined colors
    cmap=None,  # Disable colormap
    colorbar=False,  # Disable colorbar
    tiles='CartoLight',
)

# Show the plot
scatter_plot

:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Scatter.I :Scatter   [Lng]   (Lat,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [78]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[(city_data_df['Max Temp'] > 70) & (city_data_df['Max Temp'] < 80) & 
                                  (city_data_df['Wind Speed'] < 10) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [66]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tiksi,RU,71.6872,128.8694,81,
1,lazaro cardenas,MX,17.9583,-102.2000,81,
2,hasaki,JP,35.7333,140.8333,71,
3,naze,JP,28.3667,129.4833,76,
4,ilanskiy,RU,56.2358,96.0650,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [69]:
import requests
import pandas as pd
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df['Hotel Name'] = ""
radius = 10000
print("Starting hotel search")
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']
    base_url = "https://api.geoapify.com/v2/places"
    params = {
        'categories': 'accommodation.hotel',
        'filter': f'circle:{longitude},{latitude},{radius}',
        'bias': f'proximity:{longitude},{latitude}',
        'limit': 1,
        'apiKey': geoapify_key
    }
    try:
        response = requests.get(base_url, params)
        response.raise_for_status()
        name_address = response.json()
        if 'features' in name_address and name_address['features']:
            hotel_name = name_address['features'][0]['properties'].get('name', 'No hotel name available')
        else:
            hotel_name = "No hotel found"
        hotel_df.loc[index, 'Hotel Name'] = hotel_name
        print(f"{row['City']} - nearest hotel: {hotel_name}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve data for {row['City']}: {e}")
        hotel_df.loc[index, 'Hotel Name'] = "API Request Failed"
print("Hotel search completed")
print(hotel_df.head())

Starting hotel search
tiksi - nearest hotel: Арктика
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
hasaki - nearest hotel: Choshi Plaza Hotel
naze - nearest hotel: ビジネス旅館畠山
ilanskiy - nearest hotel: Берёзка
port mathurin - nearest hotel: Escale Vacances
yellowknife - nearest hotel: No hotel name available
kyaikkami - nearest hotel: ရှုခင်းသာ
antsiranana - nearest hotel: Central
korsakov - nearest hotel: гостиница "Альфа"
dudinka - nearest hotel: Виктория
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
constantia - nearest hotel: Hotel Chérica
papatowai - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
lannemezan - nearest hotel: Grand Hôtel et café de la gare
hermanus - nearest hotel: Aloe guest house
port-aux-francais - nearest hotel: Keravel
olovyannaya - nearest hotel: No hotel found
invercargill - nearest hotel: The Grand
road town - nearest hotel: No hotel name available
anadyr - nearest hotel: Гостевой дом
rongelap - nearest hot

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [70]:
%%capture --no-display

import pandas as pd
import hvplot.pandas

# Assuming you have a DataFrame called 'hotel_df' with columns 'Lat', 'Lng', 'Hotel Name', and 'Country'

# Configure the map plot with hover information
map_plot = hotel_df.hvplot.points(
    x='Lng',  # Corrected the column name from 'Lon' to 'Lng'
    y='Lat',
    geo=True,
    tiles='EsriImagery',
    frame_width=800,
    frame_height=600,
    size=10,  # Adjust the size as needed
    hover_cols=['Hotel Name', 'Country']  # Add columns to display in the hover tooltip
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)